In [ ]:
# Import modules
import os
import pandas as pd
import rdkit
# from rdkit.Chem import Draw
# from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize

# Suppress RDKit Output
rdkit.RDLogger.DisableLog('rdApp.*')

# Print versions
print(f"Pandas Version: {pd.__version__}")
print(f"RDKit Version: {rdkit.__version__}")

In [ ]:
# Paths
path_data = os.path.realpath("../data")
input_file = "chembl_approved_small_molecule_drugs.csv"
output_file = "chembl_approved_small_molecule_drugs_rdkit-profiled.csv"
input_path = os.path.join(path_data, input_file)
output_path = os.path.join(path_data, output_file)

# Load CSV
df = pd.read_csv(input_path)
df.head()

In [ ]:
largest_frag_app = rdMolStandardize.LargestFragmentChooser()
uncharge_app = rdMolStandardize.Uncharger()
mols = []
skipped_indices = []
for ii, smi in enumerate(df["smiles"]):
    try:
        mol = rdkit.Chem.MolFromSmiles(
            smi,
            sanitize = False
        )
        assert mol is not None
    except Exception as e:
        skipped_indices.append(ii)
        if pd.notna(smi):
            print(f"Error processing SMILES at index {ii}: {smi}")
            print(f"Exception: {e}")
        continue
    try:
        rdkit.Chem.SanitizeMol(mol)
        largest_frag_app.chooseInPlace(mol)
        uncharge_app.unchargeInPlace(mol)
    except Exception as e:
        skipped_indices.append(ii)
        print(f"Error processing molecule at index {ii}")
        print(f"Exception: {e}")
        continue
    descriptors = {}
    for descriptor, fxn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            value = fxn(mol)
        except Exception as e:
            print(f"Error calculating {descriptor} at index {ii}")
            print(f"Exception: {e}")
            value = "None"
        descriptors[descriptor] = value
    mols.append((ii, mol, descriptors))

print(f"{len(mols)}/{df.shape[0]} molecules processed!")
print(f"{df.shape[0]-len(mols)}/{df.shape[0]} molecules skipped!")